<a href="https://colab.research.google.com/github/JohnnyPeng123/NLP-USYD/blob/master/JPEN6856_COMP5046_Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing

## 1.1. Download Dataset

In [0]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1vF3FqgBC1Y-RPefeVmY8zetdZG1jmHzT'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_train.csv')

id = '1XhaV8YMuQeSwozQww8PeyiWMJfia13G6'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_test.csv')

import pandas as pd
df_train = pd.read_csv("imdb_train.csv")
df_test = pd.read_csv("imdb_test.csv")

reviews_train = df_train['review'].tolist()
sentiments_train = df_train['sentiment'].tolist()
reviews_test = df_test['review'].tolist()
sentiments_test = df_test['sentiment'].tolist()

print("Training set number:",len(reviews_train))
print("Testing set number:",len(reviews_test))

Training set number: 25000
Testing set number: 25000


## 1.2. Preprocess data

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word and sentiments with an integer. We'll also want to clean it up a bit before doing that.

Here are the processing steps that we'll want to take:
>
* Firstly, we will get rid of periods and extraneous punctuation -> In this case, punctuations cannot indicates whether a review is postive or negative, rather, it indicates the manutitudes of the sentiment. Thus, it's not useful for our purpose given we are not trying to predictive the magnutitudes, hence we will get rid of them.  
* Secondly, we will convert all the characters to lower-case -> Upper-case and lower-case might change the meaning of a sentence, but the overall sentiment of the reviews are unlikely to be impacted by this. Thus we should treat upper and lower words indifferently, and group them together.
* Then we will break the sentences into words and encodes them into integers.
* Then we will break the sentiments into binary numeric classes, 1 for postive and 0 for negative
* Then we will get rid of extremely long or short reviews -> the outliers

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [0]:
import re
contractions_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)
expand_contractions('You don\'t need a library')


def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]','',x)
    
    return x

In [0]:

# get rid of punctuation and convert them to lower case for training dataset
for i in range(len(reviews_train)):
  review_tmp = []
  reviews_train_temp = remove_punctuation_re(expand_contractions(reviews_train[i])).lower()
  for word in reviews_train_temp.split():
    review_tmp.append(word)
  reviews_train[i] = ' '.join(review_tmp)
  
# get rid of punctuation and convert them to lower case for test dataset
for i in range(len(reviews_test)):
  review_tmp = []
  reviews_test_temp = remove_punctuation_re(expand_contractions(reviews_test[i])).lower()
  for word in reviews_test_temp.split():
    review_tmp.append(word)
  reviews_test[i] = ' '.join(review_tmp)
  
# create a list that contains the all the words from both training and test dataset
reviews = reviews_test + reviews_train
all_text = ' '.join([w for w in reviews])
words = all_text.split()

In [0]:
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab)}

## use the dict to tokenize each words in reviews.split()
## store the tokenized reviews in reviews_ints_train & reviews_ints_test
reviews_ints_train = []
for review in reviews_train:
    reviews_ints_train.append([vocab_to_int[word] for word in review.split()])

reviews_ints_test = []
for review in reviews_test:
    reviews_ints_test.append([vocab_to_int[word] for word in review.split()])

In [0]:
# 1=positive, 0=negative label conversion
import numpy as np 

sentiments_train = np.array([1 if sentiment == 'positive' else 0 for sentiment in sentiments_train])
sentiments_test = np.array([1 if sentiment == 'positive' else 0 for sentiment in sentiments_test])

# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

In [0]:
# Please comment your code

### 2.1.1. Data Preprocessing for Word Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

**Important**: If you are going to use the code from lab3 word2vec preprocessing. Please note that `word_list = list(set(word_list)) ` has randomness. So to make sure the word_list is the same every time you run it, you can put `word_list.sort()` after that line of code.

In [0]:
# Making window size 5 skip-gram
skip_grams_target = []
skip_grams_context = []
vocab_size = len(vocab_to_int)

for i in range(5, len(words) - 5):
    # (context, target) : ([target index - 3, target index + 3], target)
    target = vocab_to_int[words[i]]
    context_temp = []
    for word in words[i - 5:i]:
        context_temp.append(vocab_to_int[word])
    for word in words[i+1:i + 6]:
        context_temp.append(vocab_to_int[word])
    context = context_temp

    # skipgrams - (target, context[0]), (target, context[1])..
    for w in context:
        skip_grams_target.append(target)
        skip_grams_context.append(w)

print(len(skip_grams_target))
print(len(skip_grams_context))

116452030
116452030


In [0]:
# Create Batches
import torch
from torch.utils.data import TensorDataset, DataLoader
batch_size = 1000

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(np.array(skip_grams_target)), torch.from_numpy(np.array(skip_grams_context)))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

### 2.1.2. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyperparameters
learning_rate = 0.005
embedding_size = 300
voc_size = len(vocab_to_int)

class SkipGram(nn.Module):
    def __init__(self):
        super(SkipGram, self).__init__()
        self.linear1 = nn.Linear(voc_size, embedding_size,bias=False)
        self.linear2 = nn.Linear(embedding_size, voc_size,bias=False)

    def forward(self, x):
        hidden = self.linear1(x)
        out = self.linear2(hidden)
        return out

skip_gram_model = SkipGram().to(device)
criterion = nn.CrossEntropyLoss() #please note we are using "CrossEntropyLoss" here
optimiser = optim.Adam(skip_gram_model.parameters(), lr=learning_rate)

### 2.1.3. Train Word Embeddings Model

In [0]:
from datetime import datetime
print(datetime.now())

data_len = len(skip_grams_target)
iter_count = 0

for epoch in range(5):
      
     for inputs, labels in train_loader:

       iter_count += 1
       inputs_list = []
       
       for i in range(len(inputs)):
        input_temp = [0]*vocab_size
        input_temp[inputs[i]] = 1
        inputs_list.append(input_temp)
       
       inputs_array = np.array(inputs_list)
       inputs_torch = torch.from_numpy(inputs_array).float().to(device)
       labels_torch = labels.to(device)
       
       skip_gram_model.train()
       # zero the parameter gradients
       optimiser.zero_grad()

       # forward + backward + optimize
       outputs = skip_gram_model(inputs_torch)
       loss = criterion(outputs, labels_torch) # We don't need to calcualte logsoftmax here
       loss.backward()
       optimiser.step()

       #if epoch % 10 == 0: 
       print('Iteration: %d, loss: %.4f' %(iter_count, loss))

print(datetime.now())

2020-04-06 02:26:08.081091
Iteration: 1, loss: 12.1050
Iteration: 2, loss: 12.1043
Iteration: 3, loss: 12.1027
Iteration: 4, loss: 12.0994
Iteration: 5, loss: 12.0945
Iteration: 6, loss: 12.0850
Iteration: 7, loss: 12.0743
Iteration: 8, loss: 12.0576
Iteration: 9, loss: 12.0386
Iteration: 10, loss: 12.0120
Iteration: 11, loss: 11.9783
Iteration: 12, loss: 11.9396
Iteration: 13, loss: 11.8654
Iteration: 14, loss: 11.8074
Iteration: 15, loss: 11.7393
Iteration: 16, loss: 11.6479
Iteration: 17, loss: 11.5357
Iteration: 18, loss: 11.4517
Iteration: 19, loss: 11.2752
Iteration: 20, loss: 11.1599
Iteration: 21, loss: 11.0341
Iteration: 22, loss: 10.8051
Iteration: 23, loss: 10.6357
Iteration: 24, loss: 10.3818
Iteration: 25, loss: 10.2235
Iteration: 26, loss: 10.0304
Iteration: 27, loss: 9.6375
Iteration: 28, loss: 9.7263
Iteration: 29, loss: 9.3786
Iteration: 30, loss: 9.0928
Iteration: 31, loss: 9.0269
Iteration: 32, loss: 9.0866
Iteration: 33, loss: 9.0329
Iteration: 34, loss: 8.9225
Iter

### 2.1.4. Save Word Embeddings Model

In [0]:
#Hint: you can refer lab1 to know how to get the weight from a Model Linear layer
weight1 = skip_gram_model.linear1.weight.data
trained_embeddings = weight1.detach().T.numpy()
np.save('word_embedding.npy',trained_embeddings)

NameError: ignored

### 2.1.5. Load Word Embeddings Model

In [0]:
# Please comment your code

## 2.2. Character Embeddings

### 2.2.1. Data Preprocessing for Character Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

In [0]:
# Please comment your code

### 2.2.2. Build Character Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.1.4. Train Character Embeddings Model

In [0]:
# Please comment your code

### 2.1.5. Save Character Embeddings Model

In [0]:
# Please comment your code

### 2.1.6. Load Character Embeddings Model

In [0]:
# Please comment your code

## 2.3. Sequence model

### 2.3.1. Apply/Import Word Embedding and Character Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.3.2. Build Sequence Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.3.3. Train Sequence Model

In [0]:
# Please comment your code

### 2.3.4. Save Sequence Model

In [0]:
# Please comment your code

### 2.3.5. Load Sequence Model

In [0]:
# Please comment your code

# 3 - Evaluation

(*Please show your empirical evidence*)

## 3.1. Performance Evaluation


You are required to provide the table with precision, recall, f1 of test set.

In [0]:
# Please comment your code

## 3.2. Hyperparameter Testing
*You are required to draw a graph(y-axis: f1, x-axis: epoch) for test set and explain the optimal number of epochs based on the learning rate you have already chosen.*

In [0]:
# Please comment your code

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed* 

In [0]:
# If you used OOP style, use this section